In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


# Clients

In [2]:
# For "registered" clients (via `@Client.register`), the client
# can be created with `create_client` by passing in the model name.
from sik_llms import create_client

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [3]:
# Or, the client can be directly instantiated
from sik_llms import OpenAI
client = OpenAI(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [4]:
# Or, the client can be directly instantiated
from sik_llms import Anthropic
client = Anthropic(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
client

# Chat

In [ ]:
from sik_llms import create_client, user_message, ResponseChunk

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously

In [6]:
client(messages=[message])

ResponseSummary(input_tokens=14, output_tokens=7, input_cost=2.1e-06, output_cost=4.2e-06, duration_seconds=0.5364799499511719, content='The capital of France is Paris.')

### Stream Asynchronously

In [7]:
responses = []
async for response in client.run_async(messages=[message]):
    if isinstance(response, ResponseChunk):
        print(response.content, end="")
        responses.append(response)

The capital of France is Paris.Warning: gpt-4o-mini may update over time. Returning num tokens assuming gpt-4o-mini-2024-07-18.


In [8]:
print(response)

input_tokens=14 output_tokens=7 input_cost=2.1e-06 output_cost=4.2e-06 duration_seconds=0.46253490447998047 content='The capital of France is Paris.'


### Claude 

In [9]:
client = create_client(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
response = client(messages=[user_message("What is the capital of France?")])
response

ResponseSummary(input_tokens=14, output_tokens=50, input_cost=4.2000000000000004e-05, output_cost=0.00075, duration_seconds=1.132070779800415, content='The capital of France is Paris. Paris is not only the capital but also the largest city in France, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.')

# OpenAI Functions/Tools

In [ ]:
from sik_llms import Tool, Parameter, RegisteredClients

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            type='string',
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_TOOLS,
    model_name='gpt-4o-mini',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=60 output_tokens=17 input_cost=9e-06 output_cost=1.0199999999999999e-05 duration_seconds=0.6613750457763672 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_9215GheoV8FAua0kPiDvg14u') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='call_9215GheoV8FAua0kPiDvg14u'


---

# Claude Functions/Tools

In [ ]:
from sik_llms import (
    create_client,
    user_message,
    Tool,
    Parameter,
    RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            type='string',
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.ANTHROPIC_TOOLS,
    model_name='claude-3-7-sonnet-latest',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=394 output_tokens=40 input_cost=0.001182 output_cost=0.0006000000000000001 duration_seconds=1.7373309135437012 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='toolu_019FRBQSVgKmNRGpd4DxfkAS') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='toolu_019FRBQSVgKmNRGpd4DxfkAS'


---

# Structured Outputs via OpenAI

In [12]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='gpt-4o-mini',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
print(response)
print('---')
print(response.content.parsed)

input_tokens=92 output_tokens=18 input_cost=1.38e-05 output_cost=1.08e-05 duration_seconds=0.6187319755554199 content=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None)
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Structured Outputs via Anthropic

In [13]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
print(response)
print('---')
print(response.content.parsed)

input_tokens=458 output_tokens=78 input_cost=0.001374 output_cost=0.00117 duration_seconds=2.8102378845214844 content=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None)
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Reasoning via OpenAI

In [14]:
from sik_llms import (
    create_client,
    user_message,
    ResponseChunk,
    ResponseSummary,
    ReasoningEffort,
)

client = create_client(
    model_name='o3-mini',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.run_async(messages=messages):
    if isinstance(response, ResponseChunk):
        print(response.content, end="")
    elif isinstance(response, ResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

First, calculate the products:
• 3 * 4 = 12
• 5 * 6 = 30

Then add them to 1 and 2:
1 + 2 + 12 + 30 = 45

The final answer is 45.

In [15]:
summary

ResponseSummary(input_tokens=28, output_tokens=57, input_cost=3.08e-05, output_cost=0.0002508, duration_seconds=2.40444016456604, content='First, calculate the products:\n• 3 * 4 = 12\n• 5 * 6 = 30\n\nThen add them to 1 and 2:\n1 + 2 + 12 + 30 = 45\n\nThe final answer is 45.')

---

# Reasoning via Claude

In [16]:
from sik_llms import (
    create_client,
    user_message,
    ResponseChunk,
    ResponseSummary,
    ReasoningEffort,
)

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None

current_type = None
async for response in client.run_async(messages=messages):
    if isinstance(response, ResponseChunk):
        if response.content_type != current_type:
            print(f"\n\n[{response.content_type.name}]")
            current_type = response.content_type
        print(response.content, end="")
    elif isinstance(response, ResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")



[THINKING]
Let me calculate 1 + 2 + (3 * 4) + (5 * 6).

First, let me evaluate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Now let me put it all together:
1 + 2 + 12 + 30 = 3 + 12 + 30 = 15 + 30 = 45

So, 1 + 2 + (3 * 4) + (5 * 6) = 45.

[TEXT]
To solve this expression, I'll work from the inside out using the order of operations:

1 + 2 + (3 * 4) + (5 * 6)

First, I'll calculate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Now I can simplify the expression:
1 + 2 + 12 + 30

Adding these numbers:
1 + 2 = 3
3 + 12 = 15
15 + 30 = 45

Therefore, 1 + 2 + (3 * 4) + (5 * 6) = 45

---